# Time Series Forecasting

## Overview

The Engine provides a simple toolset that allows users to forecast the future behavior of timeseries data using a Trainee.

## Recipe Goals

This recipe will demonstrate how to leverage Howso Engine's time series tools to forecast time-series data.

In [8]:
from IPython.display import display
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Step 1: Create Data

For this recipe, we use simulated data of a projectile that is launched vertically in the air with a randomly generated initial velocity. Then its height is calculated for each time step as gravity affects the vertical velocity until it hits the ground (height <= 0).

Additionally, we make sure that some of these series end early. We will use these unfinished series to demonstrate the ability to forecast series until their end.

In [9]:
cases = 200
max_initial_velocity = 22
min_initial_velocity = 8
dt = 0.1
gravity = 10

data = []
unfinished_ids = []

for i in range(cases):
    id = i
    dy = min_initial_velocity + np.random.random() * (max_initial_velocity - min_initial_velocity)
    time = 0
    y = 0
    stop_time = 0.75 + np.random.rand() * 2

    while y >= 0:
        data.append([id, time, y])
        dy -= 10 * dt
        y += dy
        time += dt

        if id % 20 == 0 and time > stop_time:
            unfinished_ids.append(id)
            break
    data.append([id, time, y])

df = pd.DataFrame(data, columns=['id', 'time', 'y'])

Here we plot the data. All the finished series are shown in blue and the unfinished series are shown in orange.

In [10]:
fig = go.Figure()
first_complete, first_incomplete = True, True

for id in df.id.unique():
    if id in unfinished_ids:
        fig.add_trace(go.Scattergl(
            x=df[df.id == id].time,
            y=df[df.id == id].y,
            name="Incomplete",
            mode="lines",
            line_color="darkorange",
            opacity=0.9,
            legendgroup="i",
            legendgrouptitle=dict(text="Incomplete Series"),
            showlegend=first_incomplete,
        ))
        first_incomplete=False
    else:
        fig.add_trace(go.Scattergl(
            x=df[df.id == id].time,
            y=df[df.id == id].y,
            name="Complete",
            mode="lines",
            line_color="dodgerblue",
            opacity=0.1,
            legendgroup="c",
            legendgrouptitle=dict(text="Complete Series"),
            showlegend=first_complete
        ))
        first_complete=False
fig.show()

## Step 2: Define the Feature Attributes

There are a few key steps to defining the feature attributes in this notebook:
- Confirm the data types are detected correctly
- Specify the name of both the series ID feature and the time feature
- Specify the time feature to be **not universal**

One important feature attribute in time-series flows is 'universal'. This attribute can be found within the feature attributes under the "time_series" section of the attributes of the time feature. The attribute is a boolean flag that indicates whether the time feature should be treated as "universal" or not. When the time feature is universal, the Engine will not be able to use **any** data with a time value greater than the the time value of the data being predicted, in other words, it cannot use future data. If the time feature is **not** universal, then the Engine can use all of the data seen in other series, but still cannot use future data from the series it is making a prediction on. When this attribute is unspecified, the default behavior of the Engine is to treat the time feature as if it **is universal**.

The data presented in this notebook uses a time feature that is relative to each series. Each series starts at time 0 and it increases until the series ends. The time value isn't derived from any shared context, so we can set "universal" to be false in this example. In fact, we should do so to improve accuracy. Setting "universal" to false in this setting allows the Engine to leverage more of the trained data, without leaking external information relating to time. This is why we set the 'time_feature_is_universal' flag to false in our call to `infer_feature_attributes`.

In [11]:
# Identify id-feature and time-feature
id_feature_name = "id"
time_feature_name = "time"
features = infer_feature_attributes(
    df,
    time_feature_name=time_feature_name,
    id_feature_name=id_feature_name,
    time_feature_is_universal=False,
)

features.to_dataframe()

type decimal_places     bounds                          \
                                allow_null         min         max   
id       nominal            NaN      False         NaN         NaN   
time  continuous           17.0      False    0.000000    7.089501   
y     continuous           16.0       True -180.505409  389.094000   

                               data_type id_feature original_type       \
     observed_min observed_max                          data_type size   
id            NaN          NaN    number       True       integer    8   
time     0.000000     4.300000    number        NaN       numeric    8   
y      -19.669503   228.258094    number        NaN       numeric    8   

     time_series                                                \
            type time_feature universal              delta_max   
id           NaN          NaN       NaN                    NaN   
time       delta         True     False  [0.27182818284590476]   
y           rate          NaN       NaN                    NaN   

                                                                       
                 delta_min             rate_max              rate_min  
id                     NaN                  NaN                   NaN  
time  [0.0367879441171441]                  NaN                   NaN  
y                      NaN  [567.2900066805904]  [-579.4471481199023]

## Step 3: Create the Trainee, Train, and Analyze

The one change between regular analysis using non-time series data and time series data is that during the `train` call, Engine calculates temporal information (deltas and rates) for each feature and adds these values in as additional features in your dataset.

This happens automatically if the feature attributes are configured properly.

In [12]:
t = Trainee(features=features)

# Training on the finished series data
t.train(df[~df.id.isin(unfinished_ids)])
t.analyze()

## Step 4: Forecast the series data

To forecast the series, we will use the 'continue_series' flag as well as the 'continue_series_features' and 'continue_series_values' parameters to specify the series data we would like to forecast.

In [13]:
unfinished_series_df = df[df.id.isin(unfinished_ids)]
continue_series_values = [x[1] for x in unfinished_series_df.groupby('id')]

In [14]:
synth_df = t.react_series(
    series_context_features=['id', 'time', 'y'],
    series_context_values=continue_series_values,
    action_features=['id', 'time', 'y'],
    continue_series=True,
    output_new_series_ids=False,
)['action']

display(synth_df)

,.series,id,time,y
0,series_1,0,2.7,194.043342
1,series_1,0,2.8,187.156686
2,series_1,0,2.9,179.269055
3,series_1,0,3.0,170.382202
4,series_1,0,3.1,160.496109
...,...,...,...,...
160,series_9,180,3.8,59.674031
161,series_9,180,3.9,41.752276
162,series_9,180,4.0,22.830529
163,series_9,180,4.1,2.908790


## Step 5: Inspect the Forecast Data

It's usually helpful to inspect the forecasted data, so let's inspect the visualization of the forecasted data compared to the data that was used to make the forecasts.

We hope to see smooth arcs that simulate the path of the projectile until it hits the ground.

Of course, these forecasts may not be perfect, but we should see the general shape and the series should end near when they reach zero on the Y-axis.

In [15]:
last_cases_original = df.sort_values('time', ascending=False).drop_duplicates(['id'])
synth_df_with_lasts = pd.concat([synth_df, last_cases_original]).sort_values('time', ascending=False)

In [16]:
fig = go.Figure()
first_trained = True
first_synthed = True

for id in unfinished_ids:
    fig.add_trace(go.Scattergl(
        x=df[df.id == id].time,
        y=df[df.id == id].y,
        name="Input",
        mode="lines",
        line_color="darkorange",
        opacity=0.9,
        legendgroup="i",
        legendgrouptitle=dict(text="Input Data"),
        showlegend=first_trained
    ))
    first_trained = False

    for id in synth_df_with_lasts.id.unique():
        fig.add_trace(go.Scattergl(
            x=synth_df_with_lasts[synth_df_with_lasts.id == id].time,
            y=synth_df_with_lasts[synth_df_with_lasts.id == id].y,
            name="Forecast",
            mode="lines",
            line_color="dodgerblue",
            opacity=0.1,
            legendgroup="c",
            legendgrouptitle=dict(text="Forecast Data"),
            showlegend=first_synthed
        ))
        first_synthed = False
fig.show()

## Step 6: Make a forecast with estimated uncertainty

Often when making a forecast, the user might also want to have some estimate of the uncertainty throughout the forecast.

The Howso Engine supports this through the use of the "series_residuals" detail, which returns the estimated uncertainty of
features at each timestep of the returned series. These returned values indicate the estimated uncertainty of the value in
the form of the mean absolute deviation (MAD). Thus, it is estimated that 50% of values will be within the residual returned,
75% will be within two times the residual returned, and so on.

Below we demonstrate code that plots these error bands around the given forecast. Since this recipe simulates data randomly,
it may be appropriate to rerun this cell for a better demonstration if the forecast is short or otherwise undesirable.

In [25]:
rand_id = np.random.choice(unfinished_ids)
orig_series_df = df[df.id.isin([rand_id])].sort_values('time')
continue_series_values = [unfinished_series_df[unfinished_series_df.id.isin([rand_id])].sort_values('time')]

synth_result = t.react_series(
    series_context_features=['id', 'time', 'y'],
    series_context_values=continue_series_values[:1],
    action_features=['id', 'time', 'y'],
    continue_series=True,
    output_new_series_ids=False,
    desired_conviction=10,
    details={"series_residuals": True, "series_residuals_num_samples": 50}
)

forecast_df = pd.concat([continue_series_values[0].iloc[-1:, :], synth_result['action']])
print(synth_result["details"]["series_residuals"][0])
uncertainties = [0] + [synth_result['details']['series_residuals'][0]['y']]

# Plotting
fig = go.Figure()

# The original path
fig.add_trace(go.Scattergl(
    x=orig_series_df.time,
    y=orig_series_df.y,
    name="Trained Series",
    mode="lines",
    line_color="darkorange",
    opacity=0.9,
    legendgroup="i",
    legendgrouptitle=dict(text="Input Data"),
    showlegend=True
))

# The forecast
fig.add_trace(go.Scattergl(
    x=forecast_df.time,
    y=forecast_df.y,
    name="Forecast",
    mode="lines",
    line_color="dodgerblue",
    opacity=0.9,
    legendgroup="i",
    legendgrouptitle=dict(text="Input Data"),
    showlegend=True
))

# Upper and lower band y-values for the inner 50%
lower50_uncertainties = [x - y for x, y in zip(forecast_df.y, uncertainties)]
upper50_uncertainties = [x + y for x, y in zip(forecast_df.y, uncertainties)]

# Upper and lower band y-values for the inner 75%
lower75_uncertainties = [x - 2 * y for x, y in zip(forecast_df.y, uncertainties)]
upper75_uncertainties = [x + 2 * y for x, y in zip(forecast_df.y, uncertainties)]

# The 50% band
fig.add_trace(go.Scattergl(
    x=forecast_df.time.tolist() + forecast_df.time.tolist()[::-1],
    y=upper50_uncertainties + lower50_uncertainties[::-1],
    fill='toself',
    fillcolor='rgba(0,100,80,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name="50% Confidence",
    showlegend=False
))

# The 75% band
fig.add_trace(go.Scattergl(
    x=forecast_df.time.tolist() + forecast_df.time.tolist()[::-1],
    y=upper75_uncertainties + lower75_uncertainties[::-1],
    fill='toself',
    fillcolor='rgba(100,0,80,0.15)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name="75% Confidence",
    showlegend=False
))


fig.show()

           y
0   0.045531
1   0.064354
2   0.094767
3   0.106091
4   0.113255
5   0.114339
6   0.124775
7   0.131501
8   0.139074
9   0.144556
10  0.145371
11  0.156221
12  0.166388
13  0.173696
14  0.175407
15  0.181171
16  0.183923
17  0.190898
18  0.190568
19  0.198352
20  0.202593
21  0.205201
22  0.206677
23  0.213220
24  0.216340
25  0.222082
26  0.226418
27  0.226537
28  0.227000
29  0.232450
30  0.237966


# Conclusion

In this recipe, we demonstrate how the Howso Engine can be used to forecast time-series data. Importantly, we show how to do a forecast on untrained data using `continue_series` and its related parameters. This recipe also provides generally useful information for any time-series use-case with the Howso Engine.